In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

%load_ext autoreload
%autoreload 1

from utils.utils import (
    generate_data,
    print_project_summary,
    student_assignment
)

file_location = '../data/Autumn 2024 Matching.xlsx'


In [79]:
priority_mapping = {'high': 1, 'med-high': 2, 'med': 3, 'low': 4}

max_students = {'Fermi - GNN': 3}

# Use this to force specific students onto specific projects.
forced_assignments = {}

# List of technical projects
technical_project_list = [
    'Morningstar', 'Fermi - GNN', 'Fermi - Simulations', 'Center for Living System',
    'Building Decarbonization Coalition', 'Argonne', 'CMAP', 'Invenergy'
]

priority_weights = [10, 15, 25, 30]

data_to_process_dict = generate_data(file_location, technical_project_list, priority_mapping)
all_project_list = data_to_process_dict.pop('all_project_list')
assignment_df = student_assignment(priority_weights = priority_weights, **data_to_process_dict,
                                   max_students_dict=max_students, preassigned_students=forced_assignments,
                                   number_of_projects_to_run=None, drop_projects=['IDI -- Palm Oil Concession Boundaries'], verbose=False)
print(f"\nAverage Ranking: {assignment_df.Ranking.mean():.2f}")
print(assignment_df.Ranking.value_counts())
print_project_summary(assignment_df, all_project_list)

assignment_df.to_csv('../data/test_1.csv', index=False)


Dropping 1 students because they do not have a computer and not required
Dropping 2 students because they are not high priority and not continue
Total Students available for matching: 92
1060.0

Average Ranking: 1.59
Ranking
1.0    35
2.0    19
3.0     9
Name: count, dtype: int64

Student Summary: 63 of 92 students were matched to a project.

Project Summary:


,Project Name,Number,High Priority,High-Med Priority,Med Priority,Low Priority,Experienced,Rankings,Average Ranking
0,Morningstar,4,4,0,0,0,2,"1,1,1,1",1.00
1,International Rescue Committee,4,4,0,0,0,1,"1,1,1,1",1.00
2,UChicago Library,4,3,1,0,0,2,"1,1,1,1",1.00
3,Internet Equity Initative,4,3,1,0,0,0,"1,1,1,2",1.25
4,Center for Living System,4,3,1,0,0,4,"1,1,1,2",1.25
5,Argonne,4,4,0,0,0,3,"1,1,1,2",1.25
6,Invenergy,4,4,0,0,0,2,"1,1,1,2",1.25
7,Building Decarbonization Coalition,4,2,2,0,0,2,"1,1,2,2",1.50
8,Food System 6,4,3,1,0,0,0,"1,1,2,2",1.50
9,Fermi - Simulations,4,4,0,0,0,3,"1,1,2,3",1.75


In [76]:
project_to_comp = 'CMAP'

list_of_not_assigned = list(assignment_df.loc[ (assignment_df.Priority == 2) & (assignment_df.Ranking.isna()), 'Email Address'])
prefs_df = data_to_process_dict['ranking'].copy()
prefs_df.loc[~(prefs_df.Ranking == 100.0), :].copy()

unmatched_prefs = prefs_df.loc[(prefs_df.loc[:, 'Project Name'] == project_to_comp) & prefs_df.loc[:, 'Email Address'].isin(list_of_not_assigned)
             & (prefs_df.Ranking < 100), :]

unmatched_info = pd.merge(unmatched_prefs, assignment_df.loc[:, ['Email Address', 'Experienced', 'Priority']], how='left', on='Email Address')

matched = assignment_df.loc[(assignment_df.loc[:, 'Project Assigned'] == project_to_comp), :]
display(matched.sort_values('Ranking'))

display(unmatched_info)


,Email Address,Priority,Experienced,Project Assigned,Ranking
58,mohitk@uchicago.edu,2,True,CMAP,1.0
83,xiaoyue1@uchicago.edu,2,False,CMAP,1.0
12,annapoon@uchicago.edu,1,False,CMAP,2.0
17,collinkim415@uchicago.edu,1,True,CMAP,3.0


,Email Address,Project Name,Ranking,Experienced,Priority
0,caojie2001@uchicago.edu,CMAP,5.0,False,2
1,etanios@uchicago.edu,CMAP,3.0,True,2
2,julialuo02@uchicago.edu,CMAP,4.0,True,2
3,pgajbhiye@uchicago.edu,CMAP,5.0,True,2
4,qihuiz@uchicago.edu,CMAP,5.0,False,2
5,wliao0504@uchicago.edu,CMAP,5.0,True,2
6,zhaoq@uchicago.edu,CMAP,5.0,False,2


In [11]:
data_to_process_dict['student_characteristics'].Priority.value_counts()


Priority
1    53
2    31
4     7
3     1
Name: count, dtype: int64